# Converting Scraped Data into IDPcentral Data Model and IDP Knowledge Graph

__Authors:__
Alasdair J G Gray ([ORCID:0000-0002-5711-4872](http://orcid.org/0000-0002-5711-4872)), _Heriot-Watt University, Edinburgh, UK_

Petros Papadopoulos ([ORCID:0000-0002-8110-7576](https://orcid.org/0000-0002-8110-7576)), _Heriot-Watt University, Edinburgh, UK_

Ivan Mičetić ([ORCID:0000-0003-1691-8425](https://orcid.org/0000-0003-1691-8425)), _University of Padua, Italy_

András Hatos ([ORCID:0000-0001-9224-9820](https://orcid.org/0000-0001-9224-9820)), _University of Padua, Italy_

__License:__ Apache 2.0

__Acknowledgements:__ This notebook was created during the Virtual BioHackathon-Europe 2020.

## Introduction

IDPCentral is the idea of having a central registry of proteins that are known to be disordered.

We aim to populate the content of the registry with Bioschemas markup that has been scraped using the BMUSE tool.

This notebook goes through the steps of converting the scraped content into the IDPCentral data model and also generating a Knowledge Graph from the data.

### Input

Scraped data from the following data sources:
- [DisProt](https://www.disprot.org/)
- [MobiDb](https://mobidb.bio.unipd.it/)
- [Protein Ensemble Database](https://proteinensemble.org/) (PED)

The data files can be found in this [GitHub directory](https://github.com/elixir-europe/BioHackathon-projects-2020/tree/master/projects/24/IDPCentral/scraped-data). The code in this notebook uses a relative link to retrieve the data files, i.e. `../scraped-data`.

### Output

The two generated knowledge graphs are written to file in the same directory as this notebook:
- IDPcentral
  - `IDPCentral.jsonld`: JSON-LD serialisation of the triples
  - `IDPCentral.nt`: N-TRIPLES serialisation of the triples
- IDP Knowledge Graph
  - `IDPKG.jsonld`: JSON-LD serialisation of all named graphs
  - `IDPKG.nq`: N-QUADS serialisation of all named graphs

## Code

The following code converts the data scraped using BMUSE into the desired knowledge graphs using the RDFlib python library and its abilities to process SPARQL queries over its internal data model.

The generated knowledge graph uses named graphs to track the provenance of where individual statements.

### Library Imports

In [ ]:
# Import and configure logging library
from datetime import datetime
import logging
logging.basicConfig(
    filename='idpETL.log', 
    filemode='w', 
    format='%(levelname)s:%(message)s', 
    level=logging.INFO)
logging.info('Starting processing at %s' % datetime.now().time())

In [2]:
# Imports from RDFlib
from rdflib import ConjunctiveGraph, Dataset, Graph, RDF, URIRef

In [3]:
# Import template library for templating queries
from string import Template

In [4]:
# Import functions for interacting with file directory
from glob import glob

### Template Queries

#### Provenance Query

Extracts metadata about where the data has originated and when it was scraped.

In [5]:
# Query to extract the graph and its metadata
provenanceQuery = """
PREFIX pav: <http://purl.org/pav/>
PREFIX prov: <http://www.w3.org/ns/prov#>
CONSTRUCT {
    ?g pav:retrievedFrom ?source ;
        pav:retrievedOn ?date .
}
WHERE {
    ?g pav:retrievedFrom ?source ;
        pav:retrievedOn ?date .
}
"""

#### Protein Information Query

The following query extracts data from the named graph scraped model, unifies the identifier to the UniProt accession number in the Bioschemas namespace.

Query uses `OPTIONAL` clauses throughout since it was found that not all sources comply with the minimal properties of the Bioschemas Protein Profile.

In [6]:
# Templated query for creating the direct properties for a protein entity
proteinQuery = Template("""
# Query to convert Protein scraped data to a merged named graph
# Defensive query: assumes that data does not conform to Protein profile

PREFIX bs: <https://bioschemas.org/entity/>
PREFIX pav: <http://purl.org/pav/>
PREFIX schema: <https://schema.org/>

CONSTRUCT {
    bs:${bsAccession} a schema:Protein ;
        schema:identifier ?identifier ;
        schema:name ?name ;
        schema:associatedDisease ?associatedDisease ;
        schema:description ?description ;
        schema:hasSequenceAnnotation ?annotation ;
        schema:isEncodedByBioChemEntity ?encodedBy ;
        schema:taxonomicRange ?taxonomicRange ;
        schema:url ?url ;
        schema:alternateName ?alternateName ;
        schema:bioChemInteraction ?bioChemInteraction ;
        schema:bioChemSimilarity ?bioChemSimilarity ;
        schema:hasBioChemEntityPart ?bioChemEntity ;
        schema:hasBioPloymerSequence ?sequence ;
        schema:hasMolecularFunction ?molFunction ;
        schema:hasRepresentation ?representation ;
        schema:image ?image ;
        schema:isInvolvedInBiologicalProcess ?process ;
        schema:isLocatedInSubcellularLocation ?cellularLocation ;
        schema:isPartOfBioChemEntity ?parentEntity ;
        schema:sameAs ?sameAs , ?s .
}
WHERE {
    GRAPH ?g {
# Bioschemas Minimal Properties
        ?s a schema:Protein .
        OPTIONAL {?s schema:identifier ?identifier }
        OPTIONAL {?s schema:name ?name }
## Bioschemas Recommended properties
        OPTIONAL {?s schema:associatedDisease ?associatedDisease}
        OPTIONAL {?s schema:description ?description}
        OPTIONAL {?s schema:hasSequenceAnnotation ?annotation }
        OPTIONAL {?s schema:isEncodedByBioChemEntity ?encodedBy}
        OPTIONAL {?s schema:taxonomicRange ?taxonomicRange }
        OPTIONAL {?s schema:url ?url}
## Bioschemas Optional properties
        OPTIONAL {?s schema:alternateName ?alternateName}
        OPTIONAL {?s schema:bioChemInteraction ?bioChemInteraction}
        OPTIONAL {?s schema:bioChemSimilarity ?bioChemSimilarity}
        OPTIONAL {?s schema:hasBioChemEntityPart ?bioChemEntity}
        OPTIONAL {?s schema:hasBioPloymerSequence ?sequence}
        OPTIONAL {?s schema:hasMolecularFunction ?molFunction}
        OPTIONAL {?s schema:hasRepresentation ?representation }
        OPTIONAL {?s schema:image ?image}
        OPTIONAL {?s schema:isInvolvedInBiologicalProcess ?process}
        OPTIONAL {?s schema:isLocatedInSubcellularLocation ?cellularLocation}
        OPTIONAL {?s schema:isPartOfBioChemEntity ?parentEntity}
        OPTIONAL {?s schema:sameAs ?sameAs }
    }
}
""")

#### SequenceAnnotation Query

Query to extract sequence annotations.

In [7]:
sequenceAnnotationsQuery = """
PREFIX schema: <https://schema.org/>
CONSTRUCT {
  ?s a schema:SequenceAnnotation ;
        schema:additionalProperty ?addProp ;
        schema:citation ?citation ;
        schema:creationMethod ?method ;
        schema:description ?description ;
        schema:editor ?editor ;
        schema:isPartOfBioChemEntity ?bioChemEntity ;
        schema:sequenceLocation ?seqLoc .
}
WHERE {
  graph ?g {
    ?s a schema:SequenceAnnotation .
    OPTIONAL {?s schema:additionalProperty ?addProp }
    OPTIONAL {?s schema:citation ?citation }
    OPTIONAL {?s schema:creationMethod ?method }
    OPTIONAL {?s schema:description ?description }
    OPTIONAL {?s schema:editor ?editor }
    OPTIONAL {?s schema:isPartOfBioChemEntity ?bioChemEntity }
    OPTIONAL {?s schema:sequenceLocation ?seqLoc }
  }
}
"""

#### PropertyValue Query

Query to extract PropertyValue data.

In [8]:
propertyValueQuery = """
PREFIX schema: <https://schema.org/>
CONSTRUCT {
    ?s a schema:PropertyValue ;
        schema:name ?name ;
        schema:value ?value .
}
where {
    graph ?g {
        ?s a schema:PropertyValue .
        OPTIONAL {?s schema:name ?name }
        OPTIONAL {?s schema:value ?value }
    }
}
"""

#### SequenceRange Query

Query to extract SequenceRange data.

In [9]:
sequenceRangeQuery = """
PREFIX schema: <https://schema.org/>
CONSTRUCT {
    ?s a schema:SequenceRange ;
        schema:rangeStart ?start ;
        schema:rangeEnd ?end .
}
where {
    graph ?g {
        ?s a schema:SequenceRange .
        OPTIONAL {?s schema:rangeStart ?start }
        OPTIONAL {?s schema:rangeEnd ?end}
    }
}
"""

#### UniProt ID Query

The following query extracts the UniProt IRI, declared using a `schema:sameAs` declaration in the source data. Note that different sources use alternative patterns for the UniProt IRI; the `FILTER` clause matches the following patterns
- `https://www.uniprot.org/uniprot/`
- `http://purl.uniprot.org/uniprot/`

In [10]:
# Query to extract UniProt IRI
idQuery = """
PREFIX schema: <https://schema.org/>
SELECT ?proteinIRI ?uniprot
WHERE {
    GRAPH ?g {
        ?proteinIRI a schema:Protein ;
            schema:sameAs ?uniprot .
        FILTER regex(str(?uniprot), "^(https://www|http://purl).uniprot.org/uniprot/")
    }
}
"""

#### IDPcentral Simplified Model Query

The following query transforms the data into the simplified IDPcentral JSON model.

In [11]:
# Query to convert the data to simplified IDPcentral model
idpQuery = """
PREFIX idp: <https://example.com/ipd/>
PREFIX pav: <http://purl.org/pav/>
PREFIX schema: <https://schema.org/>
CONSTRUCT {
    ?entry_url idp:name ?entry_name ;
        idp:identifier ?entry_id ;
        idp:sameAs ?uniprot_acc ;
        idp:sequence_range [
            idp:sequence_id ?sequenceID ;
            idp:start ?start ;
            idp:end ?end ;
            idp:range_annotation ?range_annotation
        ] ;
        idp:resource_name ?source ;
        idp:last_update ?date.
}
WHERE {
    GRAPH ?g {
        ?entry_url a schema:Protein ;
            schema:identifier ?entry_id ;
            schema:hasSequenceAnnotation ?sequenceID ;
            schema:sameAs ?uniprot_acc .
        OPTIONAL { 
            ?entry_url schema:name ?entry_name 
        }
        FILTER regex(str(?uniprot_acc), "^(https://www|http://purl).uniprot.org/uniprot/")
        ?sequenceID schema:sequenceLocation ?sequenceLocation .
        OPTIONAL {
            ?sequenceID schema:additionalProperty/schema:value/schema:name ?range_annotation 
        }
        ?sequenceLocation schema:rangeStart ?start ;
            schema:rangeEnd ?end.
        ?g pav:retrievedFrom ?source ;
            pav:retrievedOn ?date .
    }
}
"""

### Methods

#### Create Knowledge Graph Protein Entity 

The `createKGEntity` method uses the queries above to extract the content from the scraped data and then transform it into data using the common identifier scheme.

In [12]:
def createKGEntity(g, ds, protein, uniprot, accession):
    # Retrieve provenance of crawl and add to default graph
    result = g.query(provenanceQuery)
    # Insert provenance into default context
    for s, p, o in result:
        ds.add((s, p, o))
        # Store context of crawl
        context = (s)
    logging.debug('Context %s' % (context))
    # Parameterise the query with the proteinIRI and accession
    query = proteinQuery.substitute(proteinIRI=protein,bsAccession=accession)
    logging.debug('Query: %s' % query)
    # Create context in Dataset for the crawled entity
    ds_g = ds.graph(URIRef(context))
    # Retrieve crawled entity
    result = g.query(query)
    logging.debug("\tconvert query has %s statements." % len(result))
    # Add crawled entity to Dataset
    ds_g += result
    logging.debug('SequenceAnnotation Query: %s' % sequenceAnnotationsQuery)
    ds_g += g.query(sequenceAnnotationsQuery)
    logging.debug('PropertyValue Query: %s' % propertyValueQuery)
    ds_g += g.query(propertyValueQuery)
    logging.debug('SequenceRange Query: %s' % sequenceRangeQuery)
    ds_g += g.query(sequenceRangeQuery)

#### Create IDPcentral Entity

The following method uses the IDPcentral query to convert the harvested data into the simplified IDPcentral data model.

In [13]:
def idpExtraction(g):
    logging.debug('Query: %s' % idpQuery)
    results = g.query(idpQuery)
    logging.debug('\tQuery has %s statements.' % len(results))
    return results

#### Process Source Data Files

The following method processes the files in the given directory and calls the methods to do the data extraction.

In [14]:
def processDataFiles(idpKG, idpModel, directoryLocation):
    processed = 0
    for file in glob(directoryLocation + "*.nq"):
        logging.info("\tProcessing file: %s" % file)
        g = ConjunctiveGraph()
        g.parse(file, format="nquads")
        logging.info("\tSource has %s statements." % len(g))
        # Extract statements for IDPCentral
        idpModel += idpExtraction(g)
        logging.info("\tIDPcentral has %s statements." % len(idpModel))
        # Extract data source and UniProt IRIs
        results = g.query(idQuery)
        logging.info("\tID query result has %s statements." % len(results))
        # Convert to IDPCentral model
        for result in results:
            proteinIRI = result['proteinIRI']
            uniprotIRI = result['uniprot']
            logging.debug("\tProtein: %s\n\tUniProt: %s" % (proteinIRI, uniprotIRI))
            
            # Extract UniProt accession to use as an identifier in the Bioschemas namespace
            uniprotAccession = uniprotIRI[uniprotIRI.rindex('/')+1:]
            logging.info('Accession: %s' % uniprotAccession)
            
            # Create entity for named graph KG approach
            createKGEntity(g, idpKG, proteinIRI, uniprotIRI, uniprotAccession)
            logging.info("\tIDPKG has %s statements." % len(idpKG))
        processed += 1
    return processed

#### Output Knowledge Graphs

Writes the knowledge graphs out as n-triples and json-ld files.

In [15]:
def outputFiles(graph, label):
    # print(graph.serialize(format='nt'))
    logging.info("%s has %s statements." % (label, len(graph)))
    graph.serialize(label+'.nt', format='nt')
    graph.serialize(label+'.jsonld', format='json-ld')
    logging.info('Successfully written all triples to %s.nt' % label)

### Main Method

Processes the n-quad data files and converts them into the two knowledge graphs.

Based on the sample source data files, the following output is expected, and tested for by the assert statements.

- 8 files to have been procesed
  - 3 from DisProt
  - 2 from MobiDB
  - 3 from PED
- IDPKG should have 228 statements
  - DisProt
    - 53 after 2241.nq
    - 91 after 2243.nq
    - no additional statements from 2244.nq
  - mobiDB
    - 116 after 4283.nq
    - 134 additional statements from 5729.nq
  - PED
    - 152 statements after 6000.nq
    - 198 statements after 6001.nq
    - 222 statements after 5999.nq
- IDPCentral should have 68 statements
  - DisProt
    - 24 after 2241.nq
    - 41 after 2243.nq
    - no additional statements from 2244.nq
  - mobiDB
    - 58 after 4283.nq
    - 68 additional statements from 5729.nq
  - PED
    - 76 statements after 6000.nq
    - 112 statements after 6001.nq
    - 120 statements after 5999.nq

In [16]:
# Main control flow of the program

# Instantiate Knowledge Graphs
idpKG = Dataset()
idpModel = Graph()
totalProcessed = 0

# Process DisProt files
print("Processing DisProt...", end='')
numberOfFiles = processDataFiles(idpKG, idpModel, "../scraped-data/disprot/")
print("%d files processed" % numberOfFiles)
totalProcessed += numberOfFiles

# Process MobiDB files
print("Processing MobiDB...", end='')
numberOfFiles = processDataFiles(idpKG, idpModel, "../scraped-data/mobidb/")
print("%d files processed" % numberOfFiles)
totalProcessed += numberOfFiles

# Process PED files
print("Processing PED...", end='')
numberOfFiles = processDataFiles(idpKG, idpModel, "../scraped-data/ped/")
print("%d files processed" % numberOfFiles)
totalProcessed += numberOfFiles

# Output IDPcentral KG
outputFiles(idpModel, "IDPCentral")
# Output IDP KG
idpKG.serialize('IDPKG.nq', format='nquads')
idpKG.serialize('IDPKG.jsonld', format='json-ld')

logging.info('Processed %d files' % totalProcessed)

assert (totalProcessed == 8), "Expected 8 data files to be present!"
assert (len(idpModel) == 120), "Expected 120 statements in IDPcentral Model!"
assert (len(idpKG) == 228), "Expected 228 statements but received %r in IDP KG!" % len(idpKG)

print('\nIDPcentral has %d statements.' % len(idpModel))
print('IDP KG has %d statements.' % len(idpKG))
print('IDP KG has %d contexts.' % sum(1 for _ in idpKG.contexts()))
assert (sum(1 for _ in idpKG.contexts()) == totalProcessed), "Expect the number of contexts to be the same as the number of files processed"
print('IDP contexts:', '')
for c in idpKG.contexts():
    print('\t%s' % c)
    print('\tNumber of statements %d' % len(c))
print('\nIDP ETL process finished successfully!')

Processing DisProt...3 files processed
Processing MobiDB...2 files processed
Processing PED...3 files processed

IDPcentral has 120 statements.
IDP KG has 228 statements.
IDP KG has 8 contexts.
IDP contexts: 
	<https://bioschemas.org/crawl/v1/proteinensemble/PED00148/20201126/6000> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory'].
	Number of statements 17
	<urn:x-rdflib:default> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory'].
	Number of statements 14
	<https://bioschemas.org/crawl/v1/dev/P03265/20201108/4283> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory'].
	Number of statements 28
	<https://bioschemas.org/crawl/v1/disprot/DP00004/20201108/2243> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory'].
	Number of statements 37
	<https://bioschemas.org/crawl/v1/dev/Q12959/20201109/5729> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory'].
	Number of statements 17
	<https://bioschemas.org/crawl/v1/proteinen